In [50]:
from scipy import optimize
import numpy as np


class Nolinprog:
    """非线性规划问题类"""
    def __init__(self,
                 fun,
                 x0,
                 A=[],
                 b=[],
                 Aeq=[],
                 beq=[],
                 lb=None,
                 ub=None,
                 c=None,
                 ceq=None):

        # 初始化参数
        self.fun = fun
        self.x0 = x0
        self.A = A
        self.b = b
        self.Aeq = Aeq
        self.beq = beq
        self.lb = lb
        self.ub = ub
        self.c = c
        self.ceq = ceq
        self.cons = self.__make_cons()
        self.bounds = None

        self.x, self.fval, self.status = None, None, False

    def __make_cons(self):
        # 创建约束条件
        cons = []

        # 线性等式约束
        if np.size(self.Aeq) and np.size(self.beq):
            for i in range(self.Aeq.shape[0]):
                cons.append({
                    'type':
                    'eq',
                    'fun': (lambda x: sum(self.Aeq[i] * x) - self.beq[i])
                })

        # 线性不等式约束(>=0)
        if np.size(self.A) and np.size(self.b):
            for i in range(self.A.shape[0]):
                cons.append({
                    'type': 'ineq',
                    'fun': (lambda x: sum(self.A[i] * x) - self.b[i])
                })

        # 非线性不等式约束(>=0)
        if self.c:
            for fun in self.c():
                cons.append({'type': 'ineq', 'fun': fun})

        # 非线性等式约束
        if self.ceq:
            for fun in self.ceq():
                cons.append({'type': 'eq', 'fun': fun})

        # 上下界约束
        if not self.lb:
            self.lb = -np.ones(self.x0.shape) * np.inf

        if not self.ub:
            self.ub = np.ones(self.x0.shape) * np.inf

            self.bounds = np.array(list(zip(self.lb, self.ub)), dtype=object)

        cons = tuple(cons)
        return cons

    def solve(self):
        # 求解规划问题
        result = optimize.minimize(self.fun,
                                   self.x0,
                                   constraints=self.cons,
                                   bounds=self.bounds)
        self.x, self.fval, self.status = result.x, result.fun, result.success
        return result

    def show(self, for_min=True):
        # 展示规划结果
        if self.status == False:
            print("无满足条件的解")
            return

        if for_min:
            print(f"当x取 {self.x} 时,得最小值{self.fval}")
        else:
            print(f"当x取 {self.x} 时,得最大值{-self.fval}")

In [54]:
# 创建目标函数
def fun(x):
    y = x[0]**2 + x[1]**2 + x[2]**2 + 8
    return y


# 创建非线性不等式约束函数
def c():
    c1 = lambda x: x[0]**2 - x[1] + x[2]**2
    c2 = lambda x: -x[0] - x[1]**2 - x[2]**2 + 20
    return c1, c2


# # 创建非线性等式约束函数
def ceq():
    ceq1 = lambda x: -x[0] - x[1]**2 + 2
    ceq2 = lambda x: x[1] + 2 * x[2]**2 - 3
    return ceq1, ceq2

A=np.array([[1,1,1]])
b=np.array([3])
x0 = np.array([1, 1, 1])
lb = [0, 0, 0]

# 求解非线性规划问题
pro = Nolinprog(fun, x0,A=A,b=b, c=c, ceq=ceq, lb=lb)
pro.solve()
pro.show()

当x取 [1. 1. 1.] 时,得最小值11.0


In [41]:
np.size([])

0